In [1]:
from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

In [4]:
output_string = StringIO()

## Compostable API
with open('mike_sample.pdf', 'rb') as in_file:
    parser = PDFParser(in_file)
    doc = PDFDocument(parser)
    rsrcmgr = PDFResourceManager()
    device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    for page in PDFPage.create_pages(doc):
        interpreter.process_page(page)

s1 = output_string.getvalue()

In [ ]:
import re
amtRegex = re.compile(r'(\$)(\d{1,3})(,\d{3})?(.\d{2})?')
mo = re.findall(amtRegex, s1)

In [ ]:
numbers = []
for match in mo:
    entry = ''
    for j in match:
        if(j != '$'): entry += j
    numbers.append(entry)
numbers 

In [ ]:
for i in range(len(numbers)):
    numbers[i] = numbers[i].replace(',', '')
num_array = list(map(float, numbers))
max(num_array)

#### PDFMiner does not work

In [28]:
import pdfminer.high_level as miner
## High level API
s1 = miner.extract_text('biden_whns.pdf', page_numbers = [3])
s1

'\x0c'

In [14]:
import PyPDF2

In [15]:
obj = PyPDF2.PdfFileReader('biden_whns.pdf')

#### PyPDF2 does not work

In [30]:
obj.getPage(obj.getNumPages() - 1).extractText()

''

#### Tika

In [31]:
## Requires Java 8+ ... come back later
from tika import parser
raw = parser.from_file('biden_whns.pdf')
print(raw['content'])

2020-03-22 13:15:11,945 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.23/tika-server-1.23.jar to C:\Users\HENRYW~1\AppData\Local\Temp\tika-server.jar.
2020-03-22 13:15:50,418 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.23/tika-server-1.23.jar.md5 to C:\Users\HENRYW~1\AppData\Local\Temp\tika-server.jar.md5.
2020-03-22 13:15:51,367 [MainThread  ] [ERROR]  Unable to run java; is it installed?
2020-03-22 13:15:51,371 [MainThread  ] [ERROR]  Failed to receive startup confirmation from startServer.


RuntimeError: Unable to start Tika server.

In [2]:
import pandas as pd
df = pd.read_pickle("OPIF_04-06.pkl")

In [6]:
wmur_urls = df[df.callsign == 'WMUR-TV'].file_url
wmur_names = df[df.callsign == 'WMUR-TV'].file_name

In [7]:
len (wmur_urls)

211

In [8]:
import requests, requests_cache, time
from IPython.display import clear_output
requests_cache.install_cache()

In [30]:
n = 1
for url, name in zip(wmur_urls, wmur_names):
    print("Downloading file {}/{}".format(n,len(wmur_urls)))
    
    r = requests.get(url, stream = True)
    if (r.status_code != 200): 
        print(url, r.text)
        time.sleep(5)
    else: 
        with open('WMUR Files/' + name + '.pdf', 'wb') as f:
            f.write(r.content)
    if not getattr(r, 'from_cache', False):
        time.sleep(0.25)
    clear_output(wait = True)
    n += 1

In [29]:
import shutil
shutil.make_archive('WMUR Files', 'zip', 'WMUR Files')

'C:\\Users\\Henry Weng\\PythonNotebooks\\WMUR Files.zip'